In [1]:
!git clone https://github.com/aakashvardhan/custom-lit-yolov3

Cloning into 'custom-lit-yolov3'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 148 (delta 42), reused 124 (delta 18), pack-reused 0
Receiving objects: 100% (148/148), 2.43 MiB | 11.44 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [2]:
%cd custom-lit-yolov3

/content/custom-lit-yolov3


In [3]:

%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('/content/custom-lit-yolov3')
sys.path.append('/content/custom-lit-yolov3/dataset/PASCAL_VOC')

In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 16.8 MB/s eta 0:00:00


In [6]:
import torch
from lit_model import YOLOv3LightningModule
from lit_data_module import YOLODataModule
from pytorch_lightning import Trainer
import lightning as pl
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor, RichModelSummary,EarlyStopping
import torch
import os
import config as cfg

In [7]:
torch.cuda.empty_cache()
pl.seed_everything(1, workers=True)

INFO: Seed set to 1
INFO:lightning.fabric.utilities.seed:Seed set to 1


1

In [8]:
train_dir = '/content/custom-lit-yolov3/dataset/PASCAL_VOC/train.csv'
test_dir = '/content/custom-lit-yolov3/dataset/PASCAL_VOC/test.csv'
datamodule = YOLODataModule(config=cfg, train_dir=train_dir, test_dir=test_dir)
datamodule.setup()

ImportError: ignored

In [ ]:
model = YOLOv3LightningModule(config=cfg)

In [ ]:
from callbacks import ClassAccuracyCallback, PlotExampleCallback, MAPCallback

In [ ]:
EPOCHS = int(cfg.NUM_EPOCHS * 0.4)

In [ ]:
trainer = pl.Trainer(precision=16, max_epochs=EPOCHS,
                              callbacks=[ModelCheckpoint(dirpath="checkpoints/",verbose=True,),
                              ClassAccuracyCallback(config=cfg,train_n_epochs=1, test_n_epochs=10),
                              PlotExampleCallback(config=cfg, n_epochs = EPOCHS),
                              MAPCallback(config=cfg, test_n_epochs = EPOCHS),
                              RichModelSummary(max_depth=-1),
                              LearningRateMonitor()])

In [ ]:
from lightning.pytorch.tuner import Tuner

# Create a Tuner
tuner = Tuner(trainer)

# Finding the learning rate
lr_finder =tuner.lr_find(model,datamodule=datamodule, min_lr=1e-4, max_lr=1, num_training=trainer.max_epochs)

# Plot with
fig = lr_finder.plot(suggest=True)
fig.show()
# Pick point based on plot, or get suggestion
new_lr = lr_finder.suggestion()
print(f"{new_lr=}")